In [1]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from keras.utils import np_utils
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.layers import Bidirectional
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Add,Concatenate,BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed

from keras.engine.topology import Layer, InputSpec
from keras import initializers
from sklearn.model_selection import train_test_split

#參數
MAX_SEQUENCE_LENGTH = 512
MAX_SENTS = 15
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

MAX_SENT_LENGTH=100


column_names = ['type','title','text']
df = pd.read_csv('./all_after_mapping.tsv',sep='\t',names=column_names)
labels = df['type'].values
labels = np.array(labels)
print(labels)
# tokenlizeword = np.load('./tokenlizeword0225_nopunct.npy',allow_pickle=True)
tokenlizeword = np.load('./tokenlizeword0221.npy',allow_pickle=True)
wmodel=Word2Vec.load('./word2vec.model')

print(tokenlizeword.shape)
data=np.zeros(tokenlizeword.shape[0], MAX_SENTS, MAX_SENT_LENGTH)

Using TensorFlow backend.


2.3.1
[0 0 2 ... 3 6 2]
(35546,)


In [2]:
#x_train=文章數*最大字數*300維
x_train = []
for k in range(tokenlizeword.shape[0]):
# every article have 每篇文章最多幾個字 * 300維 embedding matrix
    embedding_matrix = np.zeros((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM))
    for i in range(len(tokenlizeword[k])):
        if(i>=MAX_SEQUENCE_LENGTH):
            break
        embedding_matrix[i] = wmodel[tokenlizeword[k][i]]
    x_train.append(embedding_matrix)

print("embedding done")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


embedding done


In [3]:
x_train = np.array(x_train)
labels = np_utils.to_categorical(labels)

dev_x = x_train[:5000]
dev_y = labels[:5000]

test_x = x_train[5000:10000]
test_y = labels[5000:10000]

train_x = x_train[10000:]
train_y = labels[10000:]
# !ls
print(dev_x.shape)
print(test_x.shape)
print(train_x.shape)

print("60%")

MemoryError: Unable to allocate 40.7 GiB for an array with shape (35546, 512, 300) and data type float64

In [ ]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
preds = Dense(2, activation='softmax')(l_lstm_sent)
model = Model(review_input, preds)